In [ ]:
import datacube
import xarray as xr
import rioxarray
import dask
from dask.distributed import performance_report
from dask.distributed import Client

from datacube.utils import masking
## https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py

%matplotlib inline
import hvplot.xarray

## Connect to ODC
dc = datacube.Datacube()

## Start local Dask cluster
## https://docs.dask.org/en/latest/setup/single-distributed.html#localcluster
n_workers = 1
n_threads = 24
client = Client(processes=False, 
                n_workers=n_workers, 
                threads_per_worker=n_threads)

In [ ]:
"""
Good subset for testing: 
x=(11.5, 11.8)
y=(50.6, 51)
"""

## Define some parameters as variables (so filenames don't need to be changed manually) 
product = "s2"
chunk_x = 3000
chunk_y = 3001

ds = dc.load(product=f"{product}_ARD",
             measurements=["pixel_qa"],
             dask_chunks={'time':-1, 'y':chunk_y, 'x':chunk_x})

ds.pixel_qa

In [ ]:
## https://force-eo.readthedocs.io/en/latest/howto/qai.html#quality-bits-in-force
# masking.describe_variable_flags(ds.pixel_qa)

flags = {'valid_data': 'valid',
        'cloud_state': 'clear',
        'cloud_shadow_flag': False}

count = masking.make_mask(ds.pixel_qa, **flags).sum(dim='time')
count

In [ ]:
%%time
with performance_report(filename=f"./reports/obs_perpixel_{product}__{n_workers}-{n_threads}_{chunk_x}-{chunk_y}.html"):
    count = count.compute()

In [ ]:
count.rio.to_raster(f"./output/obs_perpixel_{product}__{n_workers}-{n_threads}_{chunk_x}-{chunk_y}.tif", dtype="float32")